In [1]:
from src.db_management.aws_storage import SimpleStorageService
from src.constants import  *
from src.entity.config_entity import S3Config

In [2]:
s3 = SimpleStorageService(S3Config())

In [8]:
bucket = s3.get_bucket(BUCKET_NAME)

[2024-11-02 14:22:58,747: INFO: aws_storage: get bucket :: Status:Successful :: Bucket Name:wafersensorsdata]


In [4]:
s3_obj = bucket.Object("prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model_result.json")

In [5]:
s3_obj.e_tag.strip('"')

'8daf85783db476ae84b8eaa9f442b96e'

In [7]:
from pathlib import Path
Path("data/prediction_models/bestmodel_obj/Cluster_0/model.pkl")

WindowsPath('data/prediction_models/bestmodel_obj/Cluster_0/model.pkl')

In [ ]:
'8daf85783db476ae84b8eaa9f442b96e'

In [6]:
import hashlib

In [16]:
filepath = r"E:\data science\Projects\SENSOR-FAULT-DETECTION\data\best_model_result.json"

In [17]:
hasher = hashlib.md5()
with open(filepath, 'rb') as file:
    buf = file.read()
    hasher.update(buf)
print(hasher.hexdigest())

0c0d39a43184d99111c77e1abf214efd


In [5]:
import json
import hashlib
from pathlib import Path

def calculate_file_md5(file_path: Path) -> str:
    """Calculate the MD5 hash of a file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


In [ ]:
def get_s3_folder_state(bucket, prefix: str) -> dict:
    """Generate a dictionary of S3 file paths and their MD5 (ETag) hashes for a given S3 folder.

    Args:
        bucket: The S3 bucket object.
        prefix (str): The S3 folder path to scan.

    Returns:
        dict: A dictionary with S3 file keys as paths and ETag hashes.
    """
    folder_state = {}
    for obj in bucket.objects.filter(Prefix=prefix):
        if not obj.key.endswith('/'):  # Skip folders, only get files
            # Use the S3 object's ETag as an MD5-like hash
            folder_state[obj.key] = obj.e_tag.strip('"')
    return folder_state


In [ ]:
import json
from pathlib import Path

def detect_s3_folder_changes(bucket_obj, prefix: str, state_file: Path) -> bool:
    """Detect changes in the S3 folder content based on a stored state file.

    Args:
        bucket_obj: The S3 bucket object.
        prefix (str): The S3 folder path to monitor.
        state_file (Path): The path to the JSON file storing the previous state.

    Returns:
        bool: True if there are changes, False otherwise.
    """
    # Calculate current state
    current_state = get_s3_folder_state(bucket_obj, prefix)
    
    # Load previous state if exists
    if state_file.exists():
        with open(state_file, "r") as f:
            previous_state = json.load(f)
    else:
        previous_state = {}

    # Check for any changes in the folder state
    if current_state != previous_state:
        # Save the current state as the new baseline
        with open(state_file, "w") as f:
            json.dump(current_state, f, indent=4)
        return True  # Changes detected
    return False  # No changes detected


In [11]:
prefix= "prediction_model_data/champion/bestmodel_obj/"

In [17]:
state_file = Path(r"E:\data science\Projects\SENSOR-FAULT-DETECTION\notebook\etag_data.json")
state_file

WindowsPath('E:/data science/Projects/SENSOR-FAULT-DETECTION/notebook/etag_data.json')

In [20]:
with open(state_file,'r') as f:
    json.load(f)

In [25]:
detect_s3_folder_changes(bucket,prefix,state_file)

False

In [ ]:
from src.utilities.utils import read_json,

json_data = read_json(state_file)
list(json_data.values())

[2024-11-02 14:37:45,465: INFO: utils: read json data :: file_path:E:\data science\Projects\SENSOR-FAULT-DETECTION\notebook\etag_data.json :: Status:Successful]


['8daf85783db476ae84b8eaa9f442b96e',
 'cd7916157b6495f7123de59b72937c26',
 'b159fe9c485c1dd250bc58a81c80945d',
 '9e169a4a456cd678f5b89f13f32908da',
 'd6692364c4a6fc13399a0bd07bb7f7c3',
 '69e08f80a8d3a9af135a4855be4f02c1',
 '5fe31ba452a56b37890d746df0d723b3',
 'c7ccc2724858b2551596ad1a79861a9d',
 'fb07feb03a8367ec00fe9945ee117314',
 'd1b1d113685dc012ad3fec84db5b876d',
 '8daf85783db476ae84b8eaa9f442b96e',
 '0a96be1cf01d3d976d3c24d8f5e89cc1',
 '25407ee5fa2231cb80f9997b08b30840',
 '581847304499675daafb3ae9ce9e61b0',
 '259e18c3dc006f1beb0a7ff90a0e52a1']

In [12]:
for obj in bucket.objects.filter(Prefix=prefix):
    print(obj)

s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model_result.json')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_0/handle_imbalance_smote.dill')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_0/model.pkl')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_0/pca.dill')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_0/standard_scalar.dill')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_1/best_model_result.json')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champion/bestmodel_obj/Cluster_1/handle_imbalance_smote.dill')
s3.ObjectSummary(bucket_name='wafersensorsdata', key='prediction_model_data/champ

In [10]:
s3 = s3.s3_resource
bucket = s3.Bucket(BUCKET_NAME)
bucket.object_versions.delete()

[{'ResponseMetadata': {'RequestId': 'ZM2S8DD9VJES8WSQ',
   'HostId': 'CdV7A6I4QcOzPXPmMYe0ka1Gf0PjbTjrnGnlI0R2tK8N0L03rvkY5KshXLgUcfdRik4AH7a0KjsShqJsW3odaf691drfT2vT',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'CdV7A6I4QcOzPXPmMYe0ka1Gf0PjbTjrnGnlI0R2tK8N0L03rvkY5KshXLgUcfdRik4AH7a0KjsShqJsW3odaf691drfT2vT',
    'x-amz-request-id': 'ZM2S8DD9VJES8WSQ',
    'date': 'Fri, 04 Oct 2024 05:25:38 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'prediction_model_data/champion/excel_and_json_files/json_files/all_models_result.json',
    'VersionId': 'Wn9xuI8FnmIW2qCgYimwFM58fstxQNJV'},
   {'Key': 'prediction_model_data/champion/model_objs/randomforest.pkl',
    'VersionId': 'fu0R2H3XWHiJbQkG_mZHTeFtTSQozYhq'},
   {'Key': 'prediction_model_data/champion/bestmodel_obj/Cluster_1/standard_scalar.dill',
    'VersionId': 'OFy8kzgjIDCXVqjewlpaI9h.XaNbwL

In [5]:
def permanently_delete_deleted_versions(bucket_object):
    bucket = bucket_object

    # Iterate through all object versions
    for version in bucket.object_versions.all():
        if version.is_latest:
            continue  # Skip the latest version of the object
        # Permanently delete old versions and delete markers
        if version.version_id and "prediction_model_data" not in version.object_key :
            print(f"Deleting version {version.version_id} for object {version.object_key}")
            version.delete()
            
            
permanently_delete_deleted_versions(bucket)            

Deleting version tXu8HsMpFmPCyl4AIC8W62e.5OjtczLB for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_0/best_model_result.json
Deleting version BZomtOHbDoPgEsjTXcCQgBcPjHEDr3hj for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_0/handle_imbalance_smote.dill
Deleting version CrEQqhAzKn4b7W.eRFooqT2_9onpFXRA for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_0/model.pkl
Deleting version Y5PFCR9ZcS.Ux0k.1rhvUBej.ibtKE9O for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_0/pca.dill
Deleting version jnbuzRQUQcQWU.FwAJeY0FogWGASKAes for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_0/standard_scalar.dill
Deleting version kuG95S7juc9Judyl5_JaUl5yLh7_L4tp for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_1/best_model_result.json
Deleting version gZLYSMiNDy0mW0DFGKW3TM7gTn0QhiaG for object artifacts/04_10_2024_21_16_36/model_data/bestmodel_obj/Cluster_1/hand

In [8]:
%pwd

'e:\\data science\\Projects\\SENSOR-FAULT-DETECTION\\notebook'

In [9]:
os.chdir("../")

In [10]:
%pwd

'e:\\data science\\Projects\\SENSOR-FAULT-DETECTION'

In [11]:
s3.get_prediction_models(bucket)

[2024-09-18 00:08:42,369: INFO: utils: create folder using file path :: Status:Successful :: folder_path:data\prediction_models\bestmodel_obj\Cluster_0\best_model.pkl]
[2024-09-18 00:08:43,387: INFO: aws_storage: download_file_from_s3 :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: s3_file_path:prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model.pkl :: local file path:data\prediction_models/bestmodel_obj/Cluster_0/best_model.pkl ]
[2024-09-18 00:08:43,389: INFO: utils: create folder using file path :: Status:Successful :: folder_path:data\prediction_models\bestmodel_obj\Cluster_1\best_model.pkl]
[2024-09-18 00:08:45,159: INFO: aws_storage: download_file_from_s3 :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: s3_file_path:prediction_model_data/champion/bestmodel_obj/Cluster_1/best_model.pkl :: local file path:data\prediction_models/bestmodel_obj/Cluster_1/best_model.pkl ]
[2024-09-18 00:08:45,160: INFO: utils: create fold

In [12]:
os.chdir("../")

In [13]:
%pwd

'e:\\data science\\Projects\\SENSOR-FAULT-DETECTION'

In [24]:
champion_folder_path: str = "prediction_model_data/champion/"


In [25]:
champion_folder_path+"bestmodel_obj/"

'prediction_model_data/champion/bestmodel_obj/'

In [4]:
from src.entity.config_entity import S3Config
s3_config = S3Config()

In [20]:
import os
from pathlib import Path
bucket_objects = bucket.objects.filter(Prefix ="prediction_model_data/champion/bestmodel_obj/")
local_path = s3_config.local_prediction_models_path
print(local_path)
for obj in bucket_objects:
    if obj.key.endswith('.pkl'):
        s3_key = obj.key
        print(s3_key)
        local_file_path = s3_key.replace("prediction_model_data/champion",str(local_path),1)
        local_file_path = local_file_path.replace('/','\\')
        if not  os.path.exists(os.path.dirname(local_file_path)):
            os.makedirs(os.path.dirname(local_file_path),exist_ok=True)
        bucket.download_file(Key=s3_key,Filename=local_file_path)

data\prediction_models
prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model.pkl
prediction_model_data/champion/bestmodel_obj/Cluster_1/best_model.pkl
prediction_model_data/champion/bestmodel_obj/Cluster_2/best_model.pkl


In [16]:
local_path = s3_config.local_prediction_models_path

In [ ]:
import os
from pathlib import Path
bucket_objects = bucket.objects.filter(Prefix =s3_config.s3_prediction_cluster_path)
local_path = s3_config.local_prediction_models_path
print(local_path)
s3_path = s3_config.s3_prediction_cluster_path.removesuffix("/")
for obj in bucket_objects:
    if obj.key.endswith('.pkl'):
        s3_key = obj.key
        print(s3_key)
        local_file_path = s3_key.replace("prediction_model_data/champion/bestmodel_obj",str(local_path),1)
        local_file_path = local_file_path.replace('/','\\')
        if not  os.path.exists(os.path.dirname(local_file_path)):
            os.makedirs(os.path.dirname(local_file_path),exist_ok=True)
        bucket.download_file(Key=s3_key,Filename=local_file_path)

In [19]:
s3_config.s3_prediction_model_path.removesuffix('/')

'prediction_model_data/champion/bestmodel_obj'

In [6]:
from src.entity.config_entity import ModelTrainerConfig
from src.utilities.utils import get_dir_path_from_file_path,format_as_s3_path

In [37]:
path = get_dir_path_from_file_path(ModelTrainerConfig.all_model_results_json_file_path)
path

[2024-09-11 11:46:05,773: INFO: utils: get_dir_path_from_file_path :: Status:Success :: file_path:data\model_data\stable_model_data\bestmodel_obj :: folder_path:data\model_data\stable_model_data ]


WindowsPath('data/model_data/stable_model_data')

In [43]:
s3.create_s3_subfolder(bucket,path)

[2024-09-11 11:50:38,085: INFO: utils: convert path:data/model_data/stable_model_data/ to s3_path:data/model_data/stable_model_data// successfully]
[2024-09-11 11:50:39,060: INFO: aws_storage: check s3 folder exist or not :: Status: Failed :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: folder_path:data/model_data/stable_model_data//]
[2024-09-11 11:50:39,316: INFO: aws_storage: create_s3_subfolder :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: folder_path:data/model_data/stable_model_data/]


In [5]:
s3_folder_path = "client_db_data/training_batch_files/"
s3.list_files_in_s3_folder(bucket_obj=bucket,s3_folder_path=s3_folder_path)

[2024-09-11 09:46:17,339: INFO: aws_storage: list_files_in_s3_folder :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: s3_file_path:client_db_data/training_batch_files/ :: files_path:['client_db_data/training_batch_files/', 'client_db_data/training_batch_files/Wafer_07012024_000000.csv', 'client_db_data/training_batch_files/Wafer_07012024_223345.csv', 'client_db_data/training_batch_files/Wafer_08012024_120000.csv', 'client_db_data/training_batch_files/Wafer_10012024_131534.csv', 'client_db_data/training_batch_files/Wafer_11012024_151432.csv', 'client_db_data/training_batch_files/Wafer_12012024_111.csv', 'client_db_data/training_batch_files/Wafer_12012024_111213.csv', 'client_db_data/training_batch_files/Wafer_13012024_141000.csv', 'client_db_data/training_batch_files/Wafer_14012024_113045.csv', 'client_db_data/training_batch_files/Wafer_15012024_130532.csv', 'client_db_data/training_batch_files/Wafer_18012024_121532.csv', 'client_db_data/training_batch_files/Waf

['client_db_data/training_batch_files/',
 'client_db_data/training_batch_files/Wafer_07012024_000000.csv',
 'client_db_data/training_batch_files/Wafer_07012024_223345.csv',
 'client_db_data/training_batch_files/Wafer_08012024_120000.csv',
 'client_db_data/training_batch_files/Wafer_10012024_131534.csv',
 'client_db_data/training_batch_files/Wafer_11012024_151432.csv',
 'client_db_data/training_batch_files/Wafer_12012024_111.csv',
 'client_db_data/training_batch_files/Wafer_12012024_111213.csv',
 'client_db_data/training_batch_files/Wafer_13012024_141000.csv',
 'client_db_data/training_batch_files/Wafer_14012024_113045.csv',
 'client_db_data/training_batch_files/Wafer_15012024_130532.csv',
 'client_db_data/training_batch_files/Wafer_18012024_121532.csv',
 'client_db_data/training_batch_files/Wafer_19012024_141432.csv',
 'client_db_data/training_batch_files/Wafer_20012024_135132.csv',
 'client_db_data/training_batch_files/Wafer_21012024_143634.csv',
 'client_db_data/training_batch_files/

In [21]:
import boto3
import os

def download_s3_folder(bucket_name, s3_folder, local_dir):
    # Initialize S3 resource
    
    bucket = bucket_name

    # Ensure the local directory exists
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    # Download each file from the S3 folder
    for obj in bucket.objects.filter(Prefix=s3_folder):
        # Skip directories
        if obj.key.endswith('/'):
            continue
        
        # Extract the file path relative to the S3 folder
        s3_path = obj.key
        local_path = os.path.join(local_dir, os.path.relpath(s3_path, s3_folder))
        
        # Ensure the local directory structure exists
        local_dir_path = os.path.dirname(local_path)
        if not os.path.exists(local_dir_path):
            os.makedirs(local_dir_path)
        
        # Download the file
        try:
            bucket.download_file(s3_path, local_path)
            print(f'Downloaded {s3_path} to {local_path}')
        except Exception as e:
            print(f'Error downloading {s3_path}: {e}')

# Example usage
bucket_name = bucket # Replace with your actual bucket name
s3_folder = 'client_db_data/training_batch_files/'  # Ensure this ends with a slash if it's a folder
local_dir = '../Training_Batch_files'

download_s3_folder(bucket_name, s3_folder, local_dir)


Downloaded client_db_data/training_batch_files/Wafer_07012024_000000.csv to ../Training_Batch_files\Wafer_07012024_000000.csv


KeyboardInterrupt: 

In [16]:
from pathlib import Path

In [18]:
local_folder_path=Path("Training_batch_file")
os.makedirs(local_folder_path)

In [20]:
s3.download_files_from_s3(bucket_obj=bucket,local_folder_path=local_folder_path,s3_subfolder_path=s3_folder)

[2024-09-11 10:49:15,753: INFO: aws_storage: list_files_in_s3_folder :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: s3_file_path:client_db_data/training_batch_files/ :: files_path:['client_db_data/training_batch_files/', 'client_db_data/training_batch_files/Wafer_07012024_000000.csv', 'client_db_data/training_batch_files/Wafer_07012024_223345.csv', 'client_db_data/training_batch_files/Wafer_08012024_120000.csv', 'client_db_data/training_batch_files/Wafer_10012024_131534.csv', 'client_db_data/training_batch_files/Wafer_11012024_151432.csv', 'client_db_data/training_batch_files/Wafer_12012024_111.csv', 'client_db_data/training_batch_files/Wafer_12012024_111213.csv', 'client_db_data/training_batch_files/Wafer_13012024_141000.csv', 'client_db_data/training_batch_files/Wafer_14012024_113045.csv', 'client_db_data/training_batch_files/Wafer_15012024_130532.csv', 'client_db_data/training_batch_files/Wafer_18012024_121532.csv', 'client_db_data/training_batch_files/Waf

SensorFaultException: Error occurred python script name [e:\data science\projects\sensor-fault-detection\src\db_management\aws_storage.py] line number [151] error message [Error occurred python script name [e:\data science\projects\sensor-fault-detection\src\db_management\aws_storage.py] line number [162] error message [[WinError 5] Access is denied: 'Training_batch_file\\']]

In [31]:
# getting objects from s3 sub folder
objs = bucket.objects.filter(Prefix=s3_folder)
objs

s3.Bucket.objectsCollection(s3.Bucket(name='wafersensorsdata'), s3.ObjectSummary)

In [33]:
for obj in objs:
    print(obj.key)

client_db_data/training_batch_files/
client_db_data/training_batch_files/Wafer_07012024_000000.csv
client_db_data/training_batch_files/Wafer_07012024_223345.csv
client_db_data/training_batch_files/Wafer_08012024_120000.csv
client_db_data/training_batch_files/Wafer_10012024_131534.csv
client_db_data/training_batch_files/Wafer_11012024_151432.csv
client_db_data/training_batch_files/Wafer_12012024_111.csv
client_db_data/training_batch_files/Wafer_12012024_111213.csv
client_db_data/training_batch_files/Wafer_13012024_141000.csv
client_db_data/training_batch_files/Wafer_14012024_113045.csv
client_db_data/training_batch_files/Wafer_15012024_130532.csv
client_db_data/training_batch_files/Wafer_18012024_121532.csv
client_db_data/training_batch_files/Wafer_19012024_141432.csv
client_db_data/training_batch_files/Wafer_20012024_135132.csv
client_db_data/training_batch_files/Wafer_21012024_143634.csv
client_db_data/training_batch_files/Wafer_22012024_173245.csv
client_db_data/training_batch_files/

In [44]:
import os
os.path.join('data','kdd','ds')

'data\\kdd\\ds'

In [47]:
"/".join(['data',"sd","sd","hg"])+"/"

'data/sd/sd/hg/'

In [21]:
## check uploads :one file take 15sec
from pathlib import Path
path = Path("./artifacts")
path


WindowsPath('artifacts')

In [6]:
s3_path = "artifacts/"

In [10]:
import os
s3_key = os.path.join(s3_path, 'input__file.csv').replace("\\", "/")
s3_key

'artifacts/training_data/input__file.csv'

In [13]:
os.path.exists(path)

True

In [16]:
os.chdir(r"E:\data science\Projects\SENSOR-FAULT-DETECTION")

In [22]:
os.path.exists(path)

True

In [30]:
import os
for root,folder,files in os.walk(path):
    # print(root)
    # # print(folder)
    # print(files)
    if len(files)!=0:
        for file in files:
            local_file_path= os.path.join(root,file)
            s3_path = local_file_path.replace("\\","/")
            print(local_file_path)
            print(s3_path)
            print("-"*30)
            s3.upload_file_to_s3(bucket,local_file_path,s3_path)

artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_07012024_000000.csv
artifacts/09_09_2024_20_38_48/training_data/bad_raw_data/Wafer_07012024_000000.csv
------------------------------
[2024-09-12 00:29:22,308: INFO: aws_storage: upload_file_to_s3 :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: local_file_path:artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_07012024_000000.csv :: s3_file_path:artifacts/09_09_2024_20_38_48/training_data/bad_raw_data/Wafer_07012024_000000.csv ]
artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\wafer_07012024_041011.csv
artifacts/09_09_2024_20_38_48/training_data/bad_raw_data/wafer_07012024_041011.csv
------------------------------
[2024-09-12 00:29:24,110: INFO: aws_storage: upload_file_to_s3 :: Status: Successful :: Bucket_Obj:s3.Bucket(name='wafersensorsdata') :: local_file_path:artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\wafer_07012024_041011.csv :: s3_file_path:artifacts/09_09_2

In [56]:
path_d =Path(r"./artifacts/09_09_2024_20_38_48/training_data/bad_raw_data/wafer_07012024_041011.csv")
path_f = Path(r"artifacts\09_09_2024_20_38_48\training_data\bad_raw_data")
print(path_d)
print(path_f)

artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\wafer_07012024_041011.csv
artifacts\09_09_2024_20_38_48\training_data\bad_raw_data


In [62]:
os.path.isfile(path_d)

True

In [58]:
from src.utilities.utils import format_as_s3_path
format_as_s3_path(path_d)

[2024-09-12 01:48:37,549: INFO: utils: convert path:artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\wafer_07012024_041011.csv to s3_path:artifacts/09_09_2024_20_38_48/training_data/bad_raw_data/wafer_07012024_041011.csv/ successfully]


'artifacts/09_09_2024_20_38_48/training_data/bad_raw_data/wafer_07012024_041011.csv/'

In [65]:
os.path.dirname(path_f)
os.path.basename(path_f)

'bad_raw_data'

In [66]:
os.path.dirname(path_d)
os.path.basename(path_d)

'wafer_07012024_041011.csv'

In [67]:
import os

# Target path and start path
target = "/home/user/project/folder/file.txt"
start = "/home/user"

# Get relative path
relative_path = os.path.relpath(target, start)
print(relative_path)  # Output: project/folder/file.txt


project\folder\file.txt


In [68]:
import os

# When the start path is deeper than the target
target = "/home/user"
start = "/home/user/project/folder"

# Get relative path
relative_path = os.path.relpath(target, start)
print(relative_path)  # Output: ../../..


..\..


In [73]:
# Example usage
folder_path = r'artifacts\09_09_2024_20_38_48\training_data\bad_raw_data'        # Path to the folder to be zipped
output_zip_path = 'data/bad_raw_data.zip'

In [74]:
os.path.exists(output_zip_path)

True

In [77]:
import os
import zipfile

def create_zip_from_folder(folder_path, output_zip_path):
    # Create a zip file at the specified output path
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the folder, including all subdirectories and files
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Add file to the zip, preserving the folder structure
                arcname = os.path.relpath(file_path, folder_path)  # Relative path inside the zip
                zipf.write(file_path, arcname)
                print(f'Added {file_path} as {arcname}')



create_zip_from_folder(folder_path, output_zip_path)


Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_07012024_000000.csv as Wafer_07012024_000000.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\wafer_07012024_041011.csv as wafer_07012024_041011.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_07012024_223345.csv as Wafer_07012024_223345.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_08012024_120000.csv as Wafer_08012024_120000.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_10012024_131534.csv as Wafer_10012024_131534.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_11012024_151432.csv as Wafer_11012024_151432.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_12012024_111.csv as Wafer_12012024_111.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data\Wafer_12012024_111213.csv as Wafer_12012024_111213.csv
Added artifacts\09_09_2024_20_38_48\training_data\bad_raw_data

In [5]:
from src.entity.config_entity import S3Config
s3_config = S3Config()

s3_config.retraining_files_path



'artifacts/retraining_data/'

In [15]:
[file for file in bucket.objects.filter(Prefix=s3_config.retraining_files_path) if file.key!=s3_config.retraining_files_path]

[]

In [12]:
files = []
for file in dd:
    if file.key!=s3_config.retraining_files_path:
        files.append(file)

In [8]:
source_folder = "artifacts/12_09_2024_05_05_42/model_data/"
champion_folder ="prediction_model_data/champion/"
challenger_folder = "prediction_model_data/challenger/"

# check champion_folder is empty
if s3.check_s3_folder_empty(bucket,champion_folder):
    target_folder =  champion_folder
else:
    target_folder = challenger_folder    
print(target_folder)

for obj in bucket.objects.filter(Prefix=source_folder):
    source_key = obj.key
    target_key = source_key.replace(source_folder, target_folder, 1)
    print(target_key)
    # Copy the object
    s3.s3_resource.Object(BUCKET_NAME, target_key).copy_from(CopySource={'Bucket': BUCKET_NAME, 'Key': source_key})
    print(f"Copied {source_key} to {target_key}")

[2024-09-12 21:37:02,517: INFO: aws_storage: check_s3_folder_empty :: Status: Folder empty :: s3_folder_path:prediction_model_data/champion/]
prediction_model_data/champion/
prediction_model_data/champion/bestmodel_obj/best_model.dill
Copied artifacts/12_09_2024_05_05_42/model_data/bestmodel_obj/best_model.dill to prediction_model_data/champion/bestmodel_obj/best_model.dill
prediction_model_data/champion/bestmodel_obj/best_model_result.json
Copied artifacts/12_09_2024_05_05_42/model_data/bestmodel_obj/best_model_result.json to prediction_model_data/champion/bestmodel_obj/best_model_result.json
prediction_model_data/champion/cluster/cluster_obj.dill
Copied artifacts/12_09_2024_05_05_42/model_data/cluster/cluster_obj.dill to prediction_model_data/champion/cluster/cluster_obj.dill
prediction_model_data/champion/excel_and_json_files/excel_files/ALL_models_result.xlsx
Copied artifacts/12_09_2024_05_05_42/model_data/excel_and_json_files/excel_files/ALL_models_result.xlsx to prediction_model_

In [2]:
from src.utilities.utils import read_json

In [5]:
import os
from pathlib import Path
path = Path("../data/best_model_result.json")
os.path.exists(path)

True

In [6]:
best_model_result = read_json(path)


[2024-09-12 22:36:43,331: INFO: utils: read json data :: file_path:..\data\best_model_result.json :: Status:Successful]


In [29]:
i =0
for key,model_result in best_model_result.items():
    for key1,model_data in model_result.items():
        if model_data.auc_score>=0.95:
            i+=1
if  i==3:
                

In [33]:
i=0
for sc in auc_scores:
    if sc>=0.95:
        i+=1
print(i)        

3


In [26]:
best_model_result.Cluster_0

ConfigBox({'xgbclassifier': {'best_param': {'subsample': 0.9, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.2, 'gamma': 0.2, 'colsample_bytree': 0.9}, 'training_score': 1.0, 'test_score': 1.0, 'accuracy': 1.0, 'recall': 1.0, 'precision': 1.0, 'f1_score': 1.0, 'auc_score': 1.0}})

In [11]:
file_key = "prediction_models/"

In [12]:
def list_file_versions(bucket_obj, file_key):
    try:
        versions = bucket_obj.object_versions.filter(Prefix=file_key)
        
        for version in versions:
            print(f"Version ID: {version.id}, Last Modified: {version.last_modified}, Is Latest: {version.is_latest}")
    
    except Exception as e:
        print(f"Error: {e}")

list_file_versions(bucket,file_key)        

Version ID: sNMjjKMbwZetEadBjl9K2Qv4YLZf3Cwl, Last Modified: 2024-09-12 18:38:59+00:00, Is Latest: True
Version ID: 01GPE8kgdR7vgjpD7jsC4.kLJx48zTRb, Last Modified: 2024-09-12 18:39:17+00:00, Is Latest: True


In [7]:
s3.store_prediction_models()

[2024-09-12 23:48:40,131: INFO: aws_storage: get bucket :: Status:Successful :: Bucket Name:wafersensorsdata]
[2024-09-12 23:48:40,373: INFO: aws_storage: check_s3_folder_empty :: Status: Folder empty :: s3_folder_path:prediction_model_data/champion/]
[2024-09-12 23:48:40,614: INFO: aws_storage: Folder 'artifacts/12_09_2024_23_47_40/model_data/' does not exist in the bucket 's3.Bucket(name='wafersensorsdata')'.]
[2024-09-12 23:48:40,615: INFO: aws_storage: store_prediction_models execution :: Status:Failed :: source_path:artifacts/12_09_2024_23_47_40/model_data/ :: destination_path:prediction_model_data/champion/]


In [10]:
source_folder ="artifacts/13_09_2024_01_10_03/model_data/"
source_folder1 ="artifacts/13_09_2024_01_10_03/model_data/data.csv"

In [12]:
not source_folder1.endswith("/")

True

In [7]:
import os
os.chdir(r"E:\data science\Projects\SENSOR-FAULT-DETECTION")

In [8]:
s3.store_prediction_models_(source_folder)

[2024-09-13 02:25:59,021: INFO: aws_storage: get bucket :: Status:Successful :: Bucket Name:wafersensorsdata]
[2024-09-13 02:26:00,140: INFO: aws_storage: check_s3_folder_empty :: Status: Folder not empty :: s3_folder_path:prediction_model_data/champion/]
[2024-09-13 02:26:00,166: INFO: utils: read json data :: file_path:data\best_model_result.json :: Status:Successful]
[2024-09-13 02:26:00,168: INFO: utils: Cluster:0:xgbclassifier:1.0:: Status:Satisfied]
[2024-09-13 02:26:00,169: INFO: utils: Cluster:1:randomforest:1.0:: Status:Satisfied]
[2024-09-13 02:26:00,169: INFO: utils: Cluster:2:xgbclassifier:1.0:: Status:Satisfied]
[2024-09-13 02:26:00,422: INFO: aws_storage: Folder 'artifacts/13_09_2024_01_10_03/model_data/' exists in the bucket 's3.Bucket(name='wafersensorsdata')'.]
source key:artifacts/13_09_2024_01_10_03/model_data/bestmodel_obj/Cluster_0/best_model.dill
destination path: prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model.dill
[2024-09-13 02:26:01,066: INFO

In [8]:
sc_key = "artifacts/13_09_2024_01_10_03/model_data/preprocessor_stage_two/standard_scalar.dill"
sc_key.replace(source_folder,champion_folder,1)

'prediction_model_data/champion/preprocessor_stage_two/standard_scalar.dill'

In [6]:
source_folder = "artifacts/13_09_2024_01_10_03/model_data/"
champion_folder ="prediction_model_data/champion/"
challenger_folder = "prediction_model_data/challenger/"

# check champion_folder is empty
if s3.check_s3_folder_empty(bucket,champion_folder):
    target_folder =  champion_folder
else:
    target_folder = challenger_folder    
print(target_folder)

for obj in bucket.objects.filter(Prefix=source_folder):
    source_key = obj.key
    target_key = source_key.replace(source_folder, target_folder, 1)
    print(target_key)
    # Copy the object
    # s3.s3_resource.Object(BUCKET_NAME, target_key).copy_from(CopySource={'Bucket': BUCKET_NAME, 'Key': source_key})
    print(f"Copied {source_key} to {target_key}")

[2024-09-13 02:04:57,202: INFO: aws_storage: check_s3_folder_empty :: Status: Folder empty :: s3_folder_path:prediction_model_data/champion/]
prediction_model_data/champion/
prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model.dill
Copied artifacts/13_09_2024_01_10_03/model_data/bestmodel_obj/Cluster_0/best_model.dill to prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model.dill
prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model_result.json
Copied artifacts/13_09_2024_01_10_03/model_data/bestmodel_obj/Cluster_0/best_model_result.json to prediction_model_data/champion/bestmodel_obj/Cluster_0/best_model_result.json
prediction_model_data/champion/bestmodel_obj/Cluster_1/best_model.dill
Copied artifacts/13_09_2024_01_10_03/model_data/bestmodel_obj/Cluster_1/best_model.dill to prediction_model_data/champion/bestmodel_obj/Cluster_1/best_model.dill
prediction_model_data/champion/bestmodel_obj/Cluster_1/best_model_result.json
Copied artifacts/13_09_2024_